In [91]:
from bs4 import BeautifulSoup as soup
from requests import get
import pandas as pd
import numpy as np
import requests
import re
from tqdm import tqdm
import time

In [3]:
url = 'https://www.nhp.gov.in/disease-a-z/'
r = requests.get(url)
page_html = soup(r.text, 'html.parser') 
page_html

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="width=device-width, initial-scale=1, user-scalable=0" name="viewport"/>
<link href="/images/favicon-icon.ico" rel="shortcut icon" type="image/x-icon"/>
<title>
Disease / Conditions : A-Z | National Health Portal Of India        </title>
<meta content="index, follow" name="robots"/>
<link href="http://www.nhp.gov.in/disease-a-z/" rel="canonical"/> <script type="text/javascript">
        window.onload = function () {
            change();
            checkCookie();
        }

    </script>
<script type="text/javascript">
        (function (i, s, o, g, r, a, m) {
            i['GoogleAnalyticsObject'] = r; i[r] = i[r] || function () {
                (i[r].q = i[r].q || []).push(arguments)
            }, i[r].l = 1 * new Date(); a = s.createElement(o),
            m = s.getElementsByTagName(o)[0]; a.async = 1; a.src = g; m.parentNode.insertBefore(a, m)
        })(window, document, 'script', 'https://www.google-analytics.com/analytics

In [4]:
alpha = [chr(i) for i in range(ord('a'), ord('z')+1)]
diseasesUrlList=[]
for char in tqdm(alpha):
    url = 'https://www.nhp.gov.in/disease-a-z/'+char
    r = requests.get(url)
    page_html = soup(r.text, 'html.parser') 
    diseases = page_html.find('div', class_='all-disease')

    for element in diseases.find_all('a'):
        diseasesUrlList.append('https://www.nhp.gov.in' + element.get('href'))
diseasesUrlList

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:27<00:00,  1.07s/it]


['https://www.nhp.gov.in/disease/stomach/abdominal-pain',
 'https://www.nhp.gov.in/disease/digestive/liver/alcohol-abuse-and-alcoholism',
 'https://www.nhp.gov.in/disease/skin/alopecia-hair-loss',
 'https://www.nhp.gov.in/disease/neurological/anxiety',
 'https://www.nhp.gov.in/disease/digestive/appendix/appendicitis',
 'https://www.nhp.gov.in/disease/neurological/autism',
 'https://www.nhp.gov.in/disease/acquired-immunodeficiency-syndrome-aids-hiv',
 'https://www.nhp.gov.in/disease/neurological/alzheimer-s-disease',
 'https://www.nhp.gov.in/disease/blood-lymphatic/anaemia',
 'https://www.nhp.gov.in/disease/musculo-skeletal-bone-joints-/arthritis',
 'https://www.nhp.gov.in/disease/respiratory/lungs/asthma',
 'https://www.nhp.gov.in/disease/eye-ophthalmology-/airbag-eye-injury',
 'https://www.nhp.gov.in/disease/anthrax',
 'https://www.nhp.gov.in/disease/digestive/amoebiasis',
 'https://www.nhp.gov.in/disease/cardio-vascular/heart/atherosclerosis',
 'https://www.nhp.gov.in/disease/eye-oph

In [5]:
len(diseasesUrlList)

301

In [9]:
import json
diseaseData = []
with open('./Data/disease_data.json', 'w') as f:
    json.dump(diseaseData, f)

In [10]:
def write_json(new_data, filename='./Data/disease_data.json'):
    with open(filename,'r+') as file:
        file_data = json.load(file)
        file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file, indent = 4)

In [11]:
for url in tqdm(diseasesUrlList[:]):
  time.sleep(1)
  r = requests.get(url)
  data ={}
  page_html = soup(r.text, 'html.parser') 
  data['Name'] = page_html.find('h1').text.strip()
  data['Introduction'] = re.sub('(\n|\r|\t|\xa0)', '', page_html.find('div', id='Introduction').get_text())
  tabs = ['Symptoms', 'Causes', 'Diagnosis', 'Management']
  for tab in tabs:
    tab_data = page_html.find('div', id=tab)
    points = [point.get_text() for point in tab_data.find_all('li')]
    data[tab] = [re.sub('(\n|\r|\t|\xa0)', '', point) for point in points]
  write_json(data)

100%|████████████████████████████████████████████████████████████████████████████████| 301/301 [15:46<00:00,  3.15s/it]


In [68]:
df = pd.read_json('./Data/disease_data.json')
df.head()

,Name,Introduction,Symptoms,Causes,Diagnosis,Management
0,Abdominal Pain,Abdominal pain can be felt anywhere between th...,"[Sharp, stabbing cramp like pain, Brief pain t...","[Constipation, Irritable bowel syndrome, Food ...",[Abdominal discomfort that lasts 1 week or lon...,[Treatment usually is with medications for inf...
1,Alcohol Abuse and Alcoholism,Alcoholism is a condition when a person have s...,"[Continue to drink, even after knowing the ill...",[],[http://www.nlm.nih.gov],[Detoxification - It involves a nurse or docto...
2,Alopecia (hair loss),Partial or complete loss of hair is called alo...,[],[Hereditary thinning or baldness: It is also c...,[],[Minoxidil:This medicine is applied to the sca...
3,Anxiety,Anxiety is an unpleasant emotional state of un...,"[Fatigue, Dry Mouth, Abdominal cramps, Sleep d...","[Many people have anxiety, when there is chang...",[],[]
4,Appendicitis,Appendicitis is the swelling (inflammation) of...,"[Swelling in the abdomen, Loss of appetite, Na...",[http://www.nhs.uk],[http://www.nlm.nih.gov],"[http://www.nlm.nih.gov, http://www.nlm.nih.gov]"


In [138]:
def clean_data(sentence):
    sentence = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+', '', sentence)
    sentence = re.sub('www?.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|[!*.(.),]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+', '', sentence)
    return sentence

for i in df.index:
    for col in df.columns:
        if type(df[col][i]) == list:
            df[col][i] = [clean_data(sentence) for sentence in df[col][i]]
        else:
            df[col][i] = clean_data(df[col][i])
df.to_json('./Data/disease_data.json')
df = pd.read_json('./Data/disease_data.json')
df.head()

,Name,Introduction,Symptoms,Causes,Diagnosis,Management
0,Abdominal Pain,Abdominal pain can be felt anywhere between th...,"[Sharp, stabbing cramp like pain, Brief pain t...","[Constipation, Irritable bowel syndrome, Food ...",[Abdominal discomfort that lasts 1 week or lon...,[Treatment usually is with medications for inf...
1,Alcohol Abuse and Alcoholism,Alcoholism is a condition when a person have s...,"[Continue to drink, even after knowing the ill...",[],[],[Detoxification - It involves a nurse or docto...
2,Alopecia (hair loss),Partial or complete loss of hair is called alo...,[],[Hereditary thinning or baldness: It is also c...,[],[Minoxidil:This medicine is applied to the sca...
3,Anxiety,Anxiety is an unpleasant emotional state of un...,"[Fatigue, Dry Mouth, Abdominal cramps, Sleep d...","[Many people have anxiety, when there is chang...",[],[]
4,Appendicitis,Appendicitis is the swelling (inflammation) of...,"[Swelling in the abdomen, Loss of appetite, Na...",[],[],"[, ]"


In [13]:
from nltk import tokenize
from operator import itemgetter
import math
import nltk
from nltk.tokenize import word_tokenize 

In [34]:
stop_words = {"a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"}

In [101]:
def keyword_extraction(sentence):
    sentence = re.sub(r'[^\w\s]', '', sentence)
    total_words = sentence.split()
    total_word_length = len(total_words)
    #print(total_word_length)
    total_sentences = tokenize.sent_tokenize(sentence)
    #print(total_sentences)
    total_sent_len = len(total_sentences)
    #print(total_sent_len)
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in stop_words:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in stop_words:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    keywords = tf_idf_score.keys()
    return ' '.join(keywords)

In [133]:
symptoms_list = []
for i in df.index:
    symptom = ' '.join(df['Symptoms'][i])
    res = (symptom).lower()
    symptoms_list.append(keyword_extraction(res))
symptoms_list = pd.Series(symptoms_list)
symptoms_list[:5]

0                   sharp stabbing cramp pain vomiting
1    continue drink knowing ill effects drinking ho...
2                                                     
3    fatigue dry mouth abdominal cramps sleep distu...
4    swelling abdomen loss appetite nausea vomiting...
dtype: object

In [136]:
from fuzzywuzzy import process
symptom = "I got muscle pain with fever and head ache".lower()
top_10_symptom_match = process.extract(symptom, symptoms_list, limit=10)
top_10_symptom_match

[('fatigue dry mouth abdominal cramps sleep disturbances headaches muscle tension aches difficulty swallowing trembling feeling irritable twitching sweating hot flashes',
  86,
  3),
 ('swelling abdomen loss appetite nausea vomiting constipation diarrhoea inability pass gas low grade fever',
  86,
  4),
 ('headache fever sore throat muscle soreness rash mouth genital ulcers swollen lymph glands neck joint pain diarrhoea night sweats headaches blurred distorted vision cough shortness breath persistent white spots unusual lesions tongue soaking shaking chills higher 100 38 weeks chronic unexplained fatigue weight loss skin rashes',
  86,
  6),
 ('shortness breath dizziness headache coldness hands feet pale skin chest pain',
  86,
  8),
 ('difficulty walking malaise feeling tiredness weight loss poor sleep muscle aches pains tenderness moving joint',
  86,
  9),
 ('shortness breath wheezing hoarseness persistence cough worse time blood sputum pain tightening chest difficulty swallowing sw

In [137]:
diseaseList = []
for symptom in top_10_symptom_match:
    diseaseList.append(df.loc[symptom[2]].to_json()) 
diseaseList

['{"Name":"Anxiety","Introduction":"Anxiety is an unpleasant emotional state of uneasiness and distress. It is generally characterized by apprehension and worry. An anxiety disorder can have a devastating effect on the person. Most of the people go through this disease and end up in a depression.Some of the warning signals are:Intense fear or apprehensionRestlessnessEasily tiredDisturbance sleepWeight and appetite disturbancesReferences:","Symptoms":["Fatigue","Dry Mouth","Abdominal cramps","Sleep disturbances and headaches","Muscle tension and muscle aches","Difficulty swallowing","Trembling and feeling irritable","Twitching","Sweating and hot flashes"],"Causes":["Many people have anxiety, when there is change in their life like: Starting new job, getting married, Having a child, breaking up with the someone.","Some medications can also cause anxiety. These medications include inhaler medicines to treat asthma, thyroid drugs, diet pills.","Caffeine, alcohol and tobacco products may al